In [ ]:
!pip install sentence_transformers
!pip install konlpy
!pip install -U -q PyDrive
!pip install pymysql

     |████████████████████████████████| 43 kB 1.8 MB/s 


In [ ]:
import numpy as np
import itertools
import pandas as pd
from konlpy.tag import Okt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
import torch
import torch.nn as nn
import re
import pymysql

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')                      

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
cd/content/gdrive/My Drive/keybert

/content/gdrive/My Drive/keybert


In [ ]:
if torch.cuda.is_available():
  DEVICE=torch.device('cuda')
else:
  DEVICE=torch.device('cpu')

In [ ]:
model = SentenceTransformer('sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens').to(DEVICE)

In [ ]:
conn = pymysql.connect(host='mm.noye.work', user='blueturtle', password='flyturtle!', db='booka', charset='utf8mb4')
curs = conn.cursor(pymysql.cursors.DictCursor)
book_path ="Select id,title,intro,`desc`,`desc_pub`,`desc_index` From backend_book"
curs.execute(book_path)
mc=curs.fetchall()
books=pd.DataFrame(mc)

#book_path = 'booka_backend_book.csv'
#req_cols=['id','title','intro','desc','desc_pub','desc_index'] 
#books = pd.read_csv(book_path,usecols=req_cols,escapechar='\\') 
books=books.dropna(how='all')
books=books.fillna('') # nan 값 바꿔주기 
books=books[books['title'].map(len)+books['desc_pub'].map(len)+books['intro'].map(len)+books['desc'].map(len)++books['desc_index'].map(len) > 100]


books=books.reset_index()
books

,index,id,title,intro,desc,desc_pub,desc_index
0,0,2,묵향 1,,PC통신에서 폭발적인 인기를 끌며 연재되던 그 인기가 그대로 출판으로 반영된 <묵향...,,서문 : 현경과 탈마운명의 시작특이한 인물 2044호아수혈교의 출현음모(陰謀)기연(...
1,1,6,미쳐야 미친다,조선시대 지식인의 내면을 사로잡았던 열정과 광기를 탐색한 글. 남이 손가락질을 하든...,조선시대 지식인의 내면을 사로잡았던 열정과 광기를 탐색한 글. 남이 손가락질을 하든...,,1. 벽癖에 들린 사람들\n미쳐야 미친다 | 벽(癖)에 들린 사람들\n굶어 죽은 천...
2,2,8,템테이션,&lt;빅 픽처&gt;의 작가 더글라스 케네디의 장편소설. 할리우드의 영화계와 방송...,&lt;빅 픽처&gt;의 작가 더글라스 케네디의 장편소설. 한 시나리오 작가의 성공...,한 번의 성공이 반드시 ‘영원한 성공’으로 귀결되지는 않는다! \n-전 세계 30...,제1부 \n\n제2부 \n\n옮긴이의 말 \n
3,3,14,나는 왜 나를 사랑하지 못할까,독일 최고의 심리상담가 롤프 메르클레의 신작. 독일 아마존 베스트셀러 《감정사용설명...,독일 최고의 심리상담가 롤프 메르클레의 신작. 독일 아마존 베스트셀러 《감정사용설명...,나는 왜 이렇게 자존감이 낮을까?\n\n예민하고 쉽게 상처를 받는가?\n남의 눈치를...,들어가며 | 당신은 완벽하진 않지만 사랑받을 만한 사람\n\nPART 1 당신은 있...
4,4,17,삼국지 인물전,"책 속에는 총 32명의 인물이 나온다. 유표=문재인, 원술=김한길, 조자룡=조국, ...",한 눈에 파악하는 대한민국 정치 지도. 책 속에는 총 32명의 인물이 나온다. 유표...,1. &lt;삼국지 인물전&gt; 특징\n\n삼국지와 대한민국 인물이 만났다!\n안...,머리말 _ 온 나라에 썩는 냄새가 진동하는 안녕하지 못한 시대 5\n\n‘유표’ 문...
...,...,...,...,...,...,...,...
31382,32538,618440,이방인,"최연소 노벨 문학상 수상자인 카뮈의 작품으로, 출간 당시 많은 이들에게 신선한 충격...",,,
31383,32539,618604,근원의 시간 속으로,그린란드 빙하에서 지구의 숨겨진 시간을 찾아가는 한 지질학자의 깊은 사색과 기록을 ...,,,
31384,32540,618651,현대 타이베이의 탄생,"일본 제국주의 시대, 대만의 타이베이가 고유한 의미의 '장소'에서 현대 도시 '공간...",,,
31385,32541,618705,돈의 연금술,미국에서 가장 신뢰받는 돈 관리 코치이자 금융 전문가인 데이브 램지가 ‘돈의 진실’...,,,


In [ ]:
okt = Okt()
cos = nn.CosineSimilarity(dim=1, eps=1e-6)

In [ ]:
def keyword_extraction(doc,top_n):

  tokenized_doc = okt.pos(doc)
  tokenized_nouns = ' '.join([word[0] for word in tokenized_doc if word[1] == 'Noun'])
  n_gram_range = (1,1)

  count = CountVectorizer(ngram_range=n_gram_range).fit([tokenized_nouns])
  candidates = count.get_feature_names_out()

  doc_embedding = model.encode([doc])
  candidate_embeddings = model.encode(candidates)
  
  
  doc_embedding=torch.from_numpy(doc_embedding)
  candidate_embeddings=torch.from_numpy(candidate_embeddings)
  
  doc_embedding=doc_embedding.to(DEVICE)
  candidate_embeddings=candidate_embeddings.to(DEVICE)
  
  distances = cos(doc_embedding, candidate_embeddings)
  
  keywords = [candidates[index] for index in distances.argsort()[-top_n:]]
  
  return keywords

In [ ]:
book_key=dict()

In [ ]:
from tqdm import tqdm  #키워드 추출
for i in tqdm(range(len(books))):
  doc=books['desc_index'][i]+books['title'][i]+books['intro'][i]+books['desc'][i]+books['desc_pub'][i]
  if(len(re.sub('[^가-힣]', '', doc))>30):
    keyword=keyword_extraction(doc,8)
    book_key[books['id'][i]]=keyword
  else:
    continue

  0%|          | 48/31387 [00:15<2:47:51,  3.11it/s]


KeyboardInterrupt: ignored

In [ ]:
key_v=list(book_key.values())

In [ ]:
from collections import Counter # 상위 n개 키워드 출력
key_extend=[]
for i in range(len(key_v)):
  key_extend.extend(key_v[i])
count=Counter(key_extend)
count_200=count.most_common(n=200)
dict_count_200=dict((x,y)for x,y in count_200 )
dict_count_200
count_200

[('사랑', 3637),
 ('소설', 3619),
 ('작가', 2502),
 ('한국', 2382),
 ('장편소설', 1525),
 ('문학', 1264),
 ('행복', 1212),
 ('일본', 1184),
 ('베스트셀러', 1012),
 ('드라마', 971),
 ('가지', 894),
 ('대한민국', 888),
 ('독자', 885),
 ('소설가', 882),
 ('번역', 866),
 ('고통', 865),
 ('주인공', 784),
 ('연애', 779),
 ('국내', 753),
 ('조선', 729),
 ('매력', 698),
 ('가치', 686),
 ('문제', 671),
 ('이야기', 658),
 ('슬픔', 624),
 ('시인', 613),
 ('경제', 582),
 ('서울', 561),
 ('죽음', 541),
 ('독서', 539),
 ('유머', 524),
 ('애정', 518),
 ('즐거움', 512),
 ('기술', 511),
 ('기도', 486),
 ('평생', 485),
 ('학습', 484),
 ('절망', 480),
 ('욕망', 478),
 ('노하우', 474),
 ('대한', 467),
 ('데뷔', 464),
 ('결혼', 461),
 ('두려움', 460),
 ('연인', 451),
 ('활용', 450),
 ('문학상', 449),
 ('철학', 447),
 ('기쁨', 427),
 ('자존감', 408),
 ('로맨스', 405),
 ('부모', 402),
 ('독일', 397),
 ('개정판', 395),
 ('눈물', 392),
 ('한국인', 389),
 ('공부', 387),
 ('전략', 385),
 ('학교', 384),
 ('지혜', 377),
 ('추리소설', 375),
 ('살인', 372),
 ('그리움', 371),
 ('낭만', 366),
 ('열정', 355),
 ('외로움', 355),
 ('글쓰기', 349),
 ('가족', 349),
 ('전쟁', 346),
 

In [ ]:
import pickle

with open('book_key.pkl','wb') as f:
  pickle.dump(book_key,f)
with open('book_key.pkl','rb') as f:
  test = pickle.load(f)

In [ ]:
##############################################################################

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
x = np.arange(200) # 그래프
word = dict_count_200.keys()
values = dict_count_200.values()

plt.figure(figsize=(40,8))

plt.bar(x, values)
plt.xticks(x, word)

plt.show()

In [ ]:
def get_key(val): #카테고리 별 id
  matching_word=[]
  for key, value in book_key.items():
    for i in value:
      if val == i:
        matching_word.append(key)
  return matching_word

result=get_key(count_200[0][0]) 
result

In [ ]:
##################################################################################################